<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [ ]:
from glob import glob
import astropy.io.fits as fits
import json
import os, sys
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table,unique, vstack
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
from pydol.pipeline.jwst import jpipe
import multiprocessing as mp
from pydol.photometry.scripts import cmdtools
from pydol.photometry import nircam

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [ ]:
#Example data for F200W filter for NGC628
obs_table = Observations.query_criteria(proposal_id=["1783"],objectname="NGC628",
                                        filters= ['F115W', 'F150W', 'F200W'],
                                        dataRights='PUBLIC')[:3]

t = [Observations.get_product_list(obs) for obs in obs_table]
files = unique(vstack(t), keys='productFilename')

In [ ]:
mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '../data/ngc628/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files,
           dataRights = 'PUBLIC',
           productSubGroupDescription = 'UNCAL',
           download_dir = data_dir,
           flat = True,
           )""

# **Pipeline**

In [ ]:
input_files = glob(f'../data/ngc628/data/stage0/*4003*')
input_files += glob(f'../data/ngc628/data/stage0/*4004*')
out_dir     = '../data/ngc628/data'
jwst_data   = jpipe(input_files, out_dir,crds_dir='../CRDS/', filter='F115W',
                    crds_context='jwst_1303.pmap', n_cores=1)
jwst_data()

input_files = glob(f'../data/ngc628/data/stage0/*4005*')
input_files += glob(f'../data/ngc628/data/stage0/*4006*')
out_dir     = '../data/ngc628/data'
jwst_data   = jpipe(input_files, out_dir,crds_dir='../CRDS/', filter='F150W',
                    crds_context='jwst_1303.pmap', n_cores=1)
jwst_data()

input_files = glob(f'../data/ngc628/data/stage0/*4007*')
input_files += glob(f'../data/ngc628/data/stage0/*4008*')
out_dir     = '../data/ngc628/data'
jwst_data   = jpipe(input_files, out_dir,crds_dir='../CRDS/', filter='F200W',
                    crds_context='jwst_1303.pmap', n_cores=1)
jwst_data()

# **DOLPHOT Photometry**

In [ ]:
for j in ['a1', 'a2', 'a3', 'a4', 'b1', 'b2', 'b3', 'b4']:
    crf_files = glob(f'../data/ngc628/data/stage3/*{j}*_crf.fits')
    crf_files = sorted(crf_files, key=lambda x: float(x.split('/')[-1].split('_')[0][-4:])) # Sorting filters for increasing wavelength
    nircam.nircam_phot(crf_files,
               filter='f115w_f150w_f200w',
               output_dir= f'../data/ngc628/photometry/{j}',
               drz_path='../data/ngc628/photometry/F200W_step_i2d',
               cat_name= f'_{j}')

# **Photometric Analysis**

In [ ]:
Av_dict = {
            'f275w': 2.02499,
            'f336w': 1.67536,
            'f435w': 1.33879,
            'f555w': 1.03065,
            'f814w': 0.59696,

            'f090w': 0.583,
            'f115w': 0.419,
            'f150w': 0.287,
            'f200w': 0.195,

            'f438w': 1.34148,
            'f606w': 0.90941,
            'f814w': 0.59845
          }

In [ ]:
df_cmd_jwst = pd.read_csv("../data/isochrones_master/cmd_jwst.csv") # Needs to be generated

## **CMDs**

In [ ]:
ages_ = [6.8,7,7.2, 7.5,7.8,8,8.2,8.5,8.8,9,9.6,10]

ages_y = [6.8, 7.0,7.3,7.6,7.8,8,8.5,8.8]
ages_o = [9.0, 9.6,10]
mag_err_lim = 0.2

In [ ]:
tab = Table.read('../photometry/ngc628/f115w_f200w_photometry.fits')

filters = {'filt1':'f115w',
           'filt2':'f200w',
           'filt3':'f200w'}

ra_cen = 24.1738983
dec_cen = 15.7836543

positions = {'ra_col': 'ra',
             'dec_col' : 'dec',
             'ra_cen': ra_cen,
             'dec_cen': dec_cen}

region = {'r_in' : 0,
          'r_out' : 2000,
          'spatial_filter': 'circle'}

extinction = {'Av': 0.19,
              'Av_x': 3,
              'Av_y': 26,
              'Av_': 3}

axis_limits= {'xlims': [-1, 5],
              'ylims': [17.5, 28]}

isochrone_params={'met': [0.02],
                  'label_min':1,
                  'label_max':10,
                  'ages': [ 6.8,7,7.5, 8,8.5, 9]}

error_settings = {'ref_xpos': -0.25,
                  'mag_err_lim':0.2}

kde_contours = {'gen_kde':False,
                'gen_contours':False}


fig,ax, tab1 = cmdtools.gen_CMD(tab,
                      df_cmd_jwst,
                      filters,
                      positions,
                      region,
                      extinction,
                      29.83,
                      axis_limits,
                      isochrone_params,
                      kde_contours=kde_contours,
                      error_settings=error_settings,
                      other_settings = {'ab_dist':False})

isochrone_params['met'] = [0.002]
isochrone_params['ages'] = [9.1,10]

fig,ax, tab1 = cmdtools.gen_CMD(tab,
                      df_cmd_jwst,
                      filters,
                      positions,
                      region,
                      extinction,
                      29.8484,
                      axis_limits,
                      isochrone_params,
                      kde_contours=kde_contours,
                      error_settings=error_settings,
                      other_settings={'ab_dist':True, 'skip_data':True},
                      fig=fig,
                      ax=ax)